In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Content
Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

The data set includes information about:

* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

In [ ]:
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.dtypes

In [ ]:
df.columns

While the TotalCharges column should only contain the numerical values.There are some string values which are throwing error while changing the column data type.So here we are fetching the indices for those string values. 

In [ ]:
i = 0
null_str_index = []
while i<7043:
    try :
     float(df['TotalCharges'][i])
    except :
     null_str_index.append(i)
    i = i+1

Here we are inserting the NAN value in place of the string values.Later we are dropping them.

In [ ]:
for i in null_str_index :
 df['TotalCharges'][i] = np.NAN
df.dropna(inplace= True)

Now that all we have are float values.So now we can convert the column type to float.

In [ ]:
df['TotalCharges'] = df['TotalCharges'].astype(float)
df.dtypes

Here we are checking whether there are any null values in the dataset.

In [ ]:
for col in df.columns :
    print('Column ' + col )
    print(df[col].isnull().value_counts())
    

CustomerID does not contain any information so we will drop it.

In [ ]:
df.drop(columns = 'customerID',inplace = True)

Standardising these three float type features .

In [ ]:
for col in ['tenure','MonthlyCharges','TotalCharges'] :
    df[col] = df[col]/df[col].max()

Converting these feature to caategorical fetures using one hot encoding method.

In [ ]:
df_encoded = pd.get_dummies(df[['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']])

Coverting Churn values 0 for No and 1 for Yes

In [ ]:
df['Churn'].replace({'No':0,'Yes':1},inplace = True)

In [ ]:
df_pro=pd.concat([df[['MonthlyCharges','SeniorCitizen', 'TotalCharges','tenure','Churn']],df_encoded],axis =1)

In [ ]:
#df_pro[['MonthlyCharges','TotalCharges','tenure','SeniorCitizen','Contract_Month-to-month', 'Contract_One year','Contract_Two year','InternetService_DSL', 'InternetService_Fiber optic','InternetService_No','OnlineSecurity_No','OnlineSecurity_No internet service', 'OnlineSecurity_Yes','OnlineBackup_No','OnlineBackup_No internet service','DeviceProtection_No','DeviceProtection_No internet service','TechSupport_No', 'TechSupport_No internet service','PaymentMethod_Electronic check']]

In [ ]:
df_pro.columns

In [ ]:
df_pro.corr().head(5).loc['Churn']

In [ ]:
fig = plt.figure(figsize = (12,10))
ax= fig.add_subplot(111)
sns.heatmap(df_pro.corr(),ax=ax)

These all features have the highest coorelation with the Churn feature.

In [ ]:
['tenure','PaymentMethod_Electronic check','TechSupport_No','InternetService_Fiber optic','OnlineSecurity_No','Contract_Two year','PaymentMethod_Electronic check']

70% of data is for churn 'NO'.
and 30 % data is for churn 'Yes'.

In [ ]:
df['Churn'].value_counts()


* Trainng data 60%
* Cross Validation Data 20%
* Test Data 20%


In [ ]:
 train, validate, test = np.split(df_pro.sample(frac=1), [int(.6*len(df_pro)), int(.8*len(df_pro))])

In [ ]:
X_Train = train[['tenure','PaymentMethod_Electronic check','TechSupport_No','InternetService_Fiber optic','OnlineSecurity_No','Contract_Two year','PaymentMethod_Electronic check']]
Y_Train = train[['Churn']]
X_Val = validate[['tenure','PaymentMethod_Electronic check','TechSupport_No','InternetService_Fiber optic','OnlineSecurity_No','Contract_Two year','PaymentMethod_Electronic check']]
Y_Val = validate[['Churn']]
X_Test = test[['tenure','PaymentMethod_Electronic check','TechSupport_No','InternetService_Fiber optic','OnlineSecurity_No','Contract_Two year','PaymentMethod_Electronic check']]
Y_Test = test[['Churn']]

Taking a logistic regression model with L2 regularization and penalty(lamda) value as 0.1

In [ ]:
clf = LogisticRegression(penalty= 'l2' ,C = 0.1).fit(X_Train, Y_Train)
y_hat = clf.predict(X_Train)

Below is the Precision Recall curve with Average precision as 0.62

In [ ]:
disp = metrics.plot_precision_recall_curve(clf,X_Train,Y_Train)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_hat,Y_Train)
cnf_matrix

Performance on Training Set

In [ ]:
print("Precision on Train :",metrics.precision_score(y_hat,Y_Train))
print("Recall on Train :",metrics.recall_score(y_hat,Y_Train))
print("F1 Score on Train:",metrics.f1_score(y_hat,Y_Train))
print("Accuracy Score on Train :",metrics.accuracy_score(y_hat,Y_Train))

In [ ]:
y_hat_val = clf.predict(X_Val)

Performance on Validation Set

In [ ]:
print("Precision on CV :",metrics.precision_score(Y_Val,y_hat_val))
print("Recall on CV :",metrics.recall_score(Y_Val,y_hat_val))
print("F1 Score on CV :",metrics.f1_score(Y_Val,y_hat_val))
print("Accuracy Score on CV :",metrics.accuracy_score(Y_Val,y_hat_val))

In [ ]:
cnf_matrix_val = metrics.confusion_matrix(Y_Val,y_hat_val)
sns.heatmap(pd.DataFrame(cnf_matrix_val), annot=True, cmap="YlGnBu" ,fmt='g')

Performance on Test Set

In [ ]:
y_hat_test = clf.predict(X_Test)

In [ ]:
cnf_matrix_tst = metrics.confusion_matrix(Y_Test,y_hat_test)
sns.heatmap(pd.DataFrame(cnf_matrix_tst), annot=True, cmap="YlGnBu",fmt='g')

In [ ]:
print("Precision on Test :",metrics.precision_score(Y_Test,y_hat_test))
print("Recall on Test :",metrics.recall_score(Y_Test,y_hat_test))
print("F1 Score on Test :",metrics.f1_score(Y_Test,y_hat_test))
print("Accuracy Score on Test :",metrics.accuracy_score(Y_Test,y_hat_test))

On Test set the model is able to predict with around 65% confidence and has included 52% of the classes.It is still a average score.

Thanks for taking time to go through my notebook.If you have any feedback or some advice you can write it in comments section.